# Graded assignment 1 - text classification using Genetic Algorithms
## By Abdullah Karagøz

In this assignmetn we'll make a binary text classifier using genetic algorithms. We will classify movie reviews from IMDB as either negative or positive. This task consists of several steps:

1. Preprocessing of the text
2. Genetich Algorithm
3. Validation



## 1. Preprocessing

In [66]:
## Upload the text
import numpy as np
import pandas as pd
import nltk
import string
import re
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
sentence = "stayed and drinking"

lemmatizer = WordNetLemmatizer()

lemmatized = [lemmatizer.lemmatize(word) for word in word_tokenize(sentence)]
lemmatizer.lemmatize("stayed", "v")

'stay'

In [68]:
# File directories
corpus_train_pos_root = 'aclImdb/train/pos/'
corpus_train_neg_root = 'aclImdb/train/neg/'
corpus_test_pos_root = 'aclImdb/test/pos/'
corpus_test_neg_root = 'aclImdb/test/neg/'

# Corpus file objects
files_train_pos = PlaintextCorpusReader(corpus_train_pos_root, '.*')
files_train_neg = PlaintextCorpusReader(corpus_train_neg_root, '.*')
files_test_pos = PlaintextCorpusReader(corpus_test_pos_root, '.*')
files_test_neg = PlaintextCorpusReader(corpus_test_neg_root, '.*')


# Getting review texts, labels and rates all in arrays
reviews_train_pos = [files_train_pos.open(n).read() for n in files_train_pos.fileids()]
rates_train_pos = [int(re.split("_|\.", n)[-2]) for n in files_train_pos.fileids()]
labels_train_pos = [1] * len(reviews_train_pos)

reviews_train_neg = [files_train_neg.open(n).read() for n in files_train_neg.fileids()]
rates_train_neg = [int(re.split("_|\.", n)[-2]) for n in files_train_neg.fileids()]
labels_train_neg = [-1] * len(reviews_train_neg)

reviews_test_pos = [files_test_pos.open(n).read() for n in files_test_pos.fileids()]
rates_test_pos = [int(re.split("_|\.", n)[-2]) for n in files_test_pos.fileids()]
labels_test_pos = [1] * len(reviews_test_pos)

reviews_test_neg = [files_test_neg.open(n).read() for n in files_test_neg.fileids()]
rates_test_neg = [int(re.split("_|\.", n)[-2]) for n in files_test_neg.fileids()]
labels_test_neg = [-1] * len(reviews_test_neg)

In [69]:
# Putting all into two Pandas dataframes - training set and testing set
train_set = pd.DataFrame()
test_set = pd.DataFrame()

train_set['review'] = reviews_train_pos + reviews_train_neg
train_set['rate'] = rates_train_pos + rates_train_neg
train_set['label'] = labels_train_pos + labels_train_neg

test_set['review'] = reviews_test_pos + reviews_test_neg
test_set['rate'] = rates_test_pos + rates_test_neg
test_set['label'] = labels_test_pos + labels_test_neg


In [70]:
# I think to put this into own .PY file and import from there
class text_preprocessor():
    def __init__(self):
        import nltk
        import re
        import string
        
        nltk.download('stopwords')
        nltk.download('wordnet')
        from nltk.corpus import stopwords
        ", ".join(stopwords.words('english'))
        from nltk.stem.wordnet import WordNetLemmatizer 
        self.stop_words = set(stopwords.words('english'))
        
        self.punctuation = string.punctuation
        
        self.emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
        
        # src : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
        self.emoticons = {
            u":‑\)":"Happy face or smiley",
            u":\)":"Happy face or smiley",
            u":-\]":"Happy face or smiley",
            u":\]":"Happy face or smiley",
            u":-3":"Happy face smiley",
            u":3":"Happy face smiley",
            u":->":"Happy face smiley",
            u":>":"Happy face smiley",
            u"8-\)":"Happy face smiley",
            u":o\)":"Happy face smiley",
            u":-\}":"Happy face smiley",
            u":\}":"Happy face smiley",
            u":-\)":"Happy face smiley",
            u":c\)":"Happy face smiley",
            u":\^\)":"Happy face smiley",
            u"=\]":"Happy face smiley",
            u"=\)":"Happy face smiley"
        }

    def lower_case(self, text):
        return str.lower(text)
    
    def remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', self.punctuation))
    
    def remove_stopwords(self, text):
        return " ".join([word for word in str(text).split() if word not in self.stop_words])
    
    def remove_freqwords(self, text, freq_words):
        return " ".join([word for word in str(text).split() if word not in freq_words])
    
    def remove_rarewords(self, text, rare_words):
        return " ".join([word for word in str(text).split() if word not in rare_words])
    
    def remove_emoji(self, text):
        # src: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
        return self.emoji_pattern.sub(r'', text)
    
    
    def remove_emoticons(self, text):
        import re
        # src : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
        emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in self.emoticons) + u')')
        return emoticon_pattern.sub(r'', text)
    
    def convert_emoticons(self, text):
        # src : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
        for emot in self.emoticons:
            text = re.sub(u'('+emot+')', "_".join(self.emoticons[emot].replace(",","").split()), text)
        return text
    
    def remove_urls(self, text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.sub(r'', text)
        


In [71]:
# preprocessing function
def preprocess_imdb_reviews(preprocessor, df):
    df['review'] = df['review'].apply(lambda text: preprocessor.lower_case(text))
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_punctuation(text))
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_stopwords(text))
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_urls(text))
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_emoji(text))

    from collections import Counter
    cnt = Counter()
    for text in df["review"].values:
        for word in text.split():
            cnt[word] += 1

    freq_words = set([w for (w, wc) in cnt.most_common(10)])
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_freqwords(text, freq_words))

    n_rare_words = 10
    rare_words = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
    df['review'] = df['review'].apply(lambda text: preprocessor.remove_rarewords(text, rare_words))

    return df

In [124]:
preprocessor = text_preprocessor()
train_set_processed = preprocess_imdb_reviews(preprocessor, train_set)
test_set_processed = preprocess_imdb_reviews(preprocessor, test_set)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
train_set_processed

,review,rate,label
0,bromwell high cartoon comedy ran programs scho...,9,1
1,homelessness houselessness george carlin state...,8,1
2,brilliant overacting lesley ann warren best dr...,10,1
3,easily underrated inn brooks cannon sure flawe...,7,1
4,typical mel brooks much less slapstick movies ...,8,1
...,...,...,...
24995,towards end felt technical felt classroom watc...,4,-1
24996,kind enemies content watch bloody true watch m...,3,-1
24997,saw descent last night stockholm festival huge...,3,-1
24998,films pick pound turn rather 23rd century film...,1,-1


## Classifier
There are a lot of classifiers. I haven't any experience in most of them, just a little familiar with Neural Network. When assigning fitness in GA I have to test-run all the 25k samples. If I have X number of classifiers and I'll run in N number of generations, then the complexity of running the GA will be at least like:
X * N * 25,000. That'll be a big number, and my computer is not that fast.

Taking hardware limitations into account, I wanted to choose a model that's quick to run. NN with multilayers may be too complicated and too slow to run.

After taking a quick look at this link:
https://medium.com/text-classification-algorithms/text-classification-algorithms-a-survey-a215b7ab7e2d

I decided Support Vector Machine (SVM) would be best to try using as classifier as it's quick to run.






In [123]:
from collections import Counter

count_words_pos = Counter()
count_words_neg = Counter()

sum_words_pos = len(" ".join(train_set_processed.loc[train_set_processed['label'] == 1, "review"].values).split())
sum_words_neg = len(" ".join(train_set_processed.loc[train_set_processed['label'] == -1, "review"].values).split())

count_reviews_pos = Counter()
count_reviews_neg = Counter()

words_set = set((" ".join(train_set_processed['review'].values)).split())

word_freq_pos = nltk.FreqDist(" ".join(train_set_processed.loc[train_set_processed['label'] == 1, "review"].values).split())

word_freq_neg = nltk.FreqDist(" ".join(train_set_processed.loc[train_set_processed['label'] == -1, "review"].values).split())



word_weights_1 = {word: (word_freq_pos[word] / sum_words_pos) - (word_freq_neg[word] / sum_words_pos) for word in words_set}





# for text in train_set_processed.loc[train_set_processed['label'] == 1, "review"].values
#     sum_words_pos += len(text.split())
#     for word in text.split():
#         count_words_pos[word] += 1

# for text in train_set_processed.loc[train_set_processed['label'] == -1, "review"].values
#     sum_words_neg += len(text.split())
#     for word in text.split():
#         count_words_neg[word] += 1



0.0046381128849322245
review    brief prologue showing masked man stalking sla...
rate                                                      7
label                                                     1
Name: 55, dtype: object


In [144]:
def predict(text, word_weights):
    val = 0
    for word in text.split():
        if word in word_weights:
            val += word_weights[word]    
    if val > 0:
        return 1
    else:
        return -1
    
test_set_processed['preds'] = test_set_processed['review'].apply(lambda text: predict(text, word_weights_1))

correct = test_set_processed.loc[test_set_processed['preds'] == test_set_processed['label']].count()[0]

print(correct) 
print(test_set_processed['review'].size)
print(correct / test_set_processed['review'].size)

19934
25000
0.79736


19934